# Utility Bill through the lens of Climate

---
Author: Daeyoung Kim, Ellen Sun \
Date: 2023-03-30

## Introduction

For our Software Design midterm project, we decided to analyze the patterns in the giant excel sheet of utility bill of Olin through the lens of our local climate. The excel sheet contained the electricity bill from FY01 to November of FY23 with variety of information like date, total electricity consumption, time of peak demand, supply cost, demand-response, and many more. 
<p align="center">
    <img src="images/excel_sheet.png" height="500" alt="Olin's electricity bill excel sheet" />
</p>

Our central research question was:
>How does the variation in Boston climate affect Olin College’s electricity consumption?

## Tidying the Utility Data

To make our dataset useful for analysis, we need to first *tidy* the data. 

Currently, the spreadsheet has rows as variables and columns as observation. The observations are also spread across different tabs by their fiscal year. There are empty rows in the sheet as well as non-uniform nomenclature. The format of values are not standarized and vary across cells within the rows. Thus, the data right now is not very useful to us.  

There are 3 rules to tidy data:
>1. Every column is a variable.
>2. Every row is an observation.
>3. Every cell is a single value.

[Source](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html)

We processed the dataset to adhere to the 3 rules for convenient analysis. 

Below is the code to autoreload cells for our own convenience.

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from os import path
from functions_manage_data import tidy_data

PATH_UTILITY = "data/electricity_FY13_23.csv"
PATH_UTILITY_JITTERED = "data/electricity_FY13_23_jittered.csv"

if path.isfile(PATH_UTILITY) or path.isfile(PATH_UTILITY_JITTERED):
    print("The utility data is ready!")
else:
    tidy_data("data/FY23_Electric_Data_Campus_Houses_edited.xlsx")

The utility data is ready!


Here's a sample of how it looked after processing:

In [72]:
import pandas as pd
from IPython.display import HTML

try:
    df_util = pd.read_csv(PATH_UTILITY)
except FileNotFoundError:
    df_util = pd.read_csv(PATH_UTILITY_JITTERED)

display(HTML(df_util.head(10).to_html()))

,start_read_date,end_read_date,total_consumption,time_of_peak_demand,total_cost
0,2013-04-05 00:00:00,2013-05-06 00:00:00,337440,14:30:00,52000.85
1,2013-05-06 00:00:00,2013-06-05 00:00:00,316800,14:55:00,50335.03
2,2013-06-05 00:00:00,2013-07-07 00:00:00,404640,12:50:00,59092.46
3,2013-07-07 00:00:00,2013-08-05 00:00:00,424320,12:10:00,64701.47
4,2013-08-05 00:00:00,2013-09-03 00:00:00,350400,11:15:00,49643.51
5,2013-09-03 00:00:00,2013-10-03 00:00:00,354720,13:35:00,52519.41
6,2013-10-03 00:00:00,2013-11-04 00:00:00,338880,13:30:00,41734.16
7,2013-11-04 00:00:00,2013-12-05 00:00:00,299520,11:10:00,39356.74
8,2013-12-05 00:00:00,2014-01-06 00:00:00,276960,16:55:00,51161.32
9,2014-01-06 00:00:00,2014-02-04 00:00:00,259680,14:15:00,55521.64


## Gathering Climate Data
In order to answer our research question, we used the publicly available [National Centers for Environmental Information's Climate Data Online (CDO) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) to gather Boston's climate data. We chose Boston as our source of 'local' climate since it was geographically proximal to Olin College and had the most comprehensive data because of Boston Logan Airport's stable data collection. 

The employed steps in gathering data from the API were:
1. Find the Massachusetts ID used in the API
2. Find weather stations in MA
3. Choose a station and find available datatypes
4. Choose relevant datatypes and fetch data
5. Save into a .json and pandas dataframe

In [73]:
import requests

FIPS_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=ST&limit=52"

with open("API_KEY.txt", "r", encoding="utf-8") as file:
    token = file.read()

response = requests.get(FIPS_URL, headers={"token": token})
print(response.json()["results"][21])

KeyboardInterrupt: 

Now, using `FIPS:25` as ID for Massachusetts, we found all the available stations. We found the ID (`GHCND:USW00014739`) for the Boston Logan weather station, which was the station that had the most coverage and would work best for our needs.

In [ ]:
import json

STATION_URL = (
    "https://www.ncei.noaa.gov/cdo-web/api/v2/stations/GHCND:USW00014739"
)

response = requests.get(STATION_URL, headers={"token": token})
boston_logan_airport = json.dumps(response.json(), indent=4)
print(boston_logan_airport)

In [ ]:
START_DATE = "2013-04-01"
END_DATE = "2022-12-31"
DATATYPE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/datatypes?stationid=GHCND:USW00014739&startdate=2013-04-01&enddate=2022-12-31"
response = requests.get(DATATYPE_URL, headers={"token": token})
df_datatypes = pd.json_normalize(response.json(), record_path=["results"])

Here were some of the available datatypes:

In [ ]:
display(HTML(df_datatypes.head(10).to_html()))

Although our intial approach was using the API for listing the available datatypes, the results were unorganized even with start and end date queries and did not give us the information that we needed. Thus, we found an [online documentation page](https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00014739/detail) of the specific weather station and looked for available datatypes there. 
<p align="center">
    <img src="images/station.png" height="500" alt="Station documentation page" />
</p>

We chose `TAVG`, `AWND`, and `PRCP` for the datatypes and saved them locally as .json files. 

In [ ]:
from os import path
from functions_manage_data import get_data_api

DATATYPES = ["TAVG", "PRCP", "AWND"]

for datatype in DATATYPES:
    if path.isfile(f"data/{datatype}.json"):
        print(f"{datatype} dataset is ready!")
    else:
        get_data_api(datatype)

After storing the information about each datatype into a `.json` file, we flattened the `.json` files into `pandas` dataframes. Then, we read the utilities data into a pandas dataframe and created a joint dataframe between each weather datatype and the utility information to make it easier to plot. Below is a sample of how the dataframe containing information about the average wind speed looks like. 

In [ ]:
from functions_manage_data import flatten_json, join_dataframes

df_avg_temp = flatten_json("TAVG")
df_total_prcp = flatten_json("PRCP")
df_avg_wind = flatten_json("AWND")

df_util_wind = join_dataframes(df_avg_wind, df_util)
df_util_temp = join_dataframes(df_avg_temp, df_util)
df_util_prcp = join_dataframes(df_total_prcp, df_util)

display(HTML(df_util_wind.head(10).to_html()))

The joint dataframes contain columns detailing utility information (`total_consumption`, `time_of_peak_demand`, `total_cost`) and weather information (`datatype`, showing which category this data is from, and `value`, showing the actual value taken-- in the displayed case, the average monthly wind speed in MPH)

Now we can plot each weather datatype against the `total monthly electricity consumption` (kwh) of Olin.  

In [ ]:
from functions_plot_data import plot_weather_util_2_plots

plot_weather_util_2_plots(df_util_temp)
plot_weather_util_2_plots(df_util_prcp)
plot_weather_util_2_plots(df_util_wind)

Next, we put each specific `Pearson's correlation coefficient` in a table to see the exact correlations between each of the weather datatypes and total monthly electricity consumption:

In [ ]:
from functions_manage_data import calc_weather_util_corr
df_correlation = pd.DataFrame(columns=["weather_datatype", "pearson_r_value"])

df_util_weathers = [df_util_temp, df_util_prcp, df_util_wind]

for i, df_util_weather in enumerate(df_util_weathers):
    df_correlation.at[i, "weather_datatype"] = df_util_weather["datatype"][0]
    df_correlation.at[i, "pearson_r_value"] = calc_weather_util_corr(df_util_weather)

display(HTML(df_correlation.to_html()))

From these graphs and the table, we noticed three interesting patterns:
- `Average Temperature` had a `highly positive` correlation with electricity consumption
- `Average Wind Speed` had a `fairly negative` correlation with electricity consumption 
- Olin's electricity consumption seemed to rise significantly around 2016-2017, which has no apparent cause from any of these three weather patterns.
    
### Temperature vs Consumption

Firstly, we hypothesized that temperature might be correlated with electricity consumption because Olin uses more air conditioning in the summer and turns it off in the winter. To check this, we created box plots to see the consumption for each of the seasons. The seasons were sorted by month, with December - Feburary being winter, March - May being spring, June - August being summer, and September - November being fall.

In [ ]:
import matplotlib.pyplot as plt
from functions_manage_data import filter_season

SEASON_NAMES = ["Winter", "Spring", "Summer", "Fall"]

season_dfs = []
for season in SEASON_NAMES:
    df = filter_season(season, df_util_temp)
    season_dfs.append(df.reset_index())

season_data = []
for df in season_dfs:
    season_data.append(df["total_consumption"])

plt.boxplot(season_data, labels=SEASON_NAMES)
plt.ylabel("Total Energy Consumption (kwh)")
plt.title("Total Energy Consumption in each Season (kwh)")
plt.show()

The boxplot above shows the ranges and medians of electricity consumption for each of the seasons. As we expected, `winter` had significantly `lower consumption` while `summer` had significantly `higher` values. This information supports our hypothesis that the high correlation between temperature and electricity consumption was indeed because of how Olin manages its power usage during the winter and summer seasons. 

### Wind Speed vs Consumption (vs Average Temperature)

While the connection between temperature and electricity consumption might be intuitive, the correlation found for average wind speed is not. To explore more into this phenomenon, we graphed `wind speed` against `average temperature`.

In [ ]:
import numpy as np

avg_temp = df_util_temp["value"]
avg_wind = df_util_wind["value"]
plt.scatter(avg_temp, avg_wind, color="blue")
plt.xlabel("Average Monthly Temperature (F°)")
plt.ylabel("Average Monthly Wind Speed (mph)")
plt.title("Average Monthly Wind Speed vs Average Monthly Temperature")

z = np.polyfit(avg_temp, avg_wind, 1)
p = np.poly1d(z)
plt.plot(avg_temp, p(avg_temp), color="red", label="Best Fit Line")
plt.legend()
plt.show()

temp_wind_corr = np.corrcoef(avg_temp, avg_wind)[0][1]
print(f"The Pearson correlation coefficient is {temp_wind_corr}")

Looking at the graph and the correlation coefficient (`-0.71`) above, we can see that average wind speed is clearly `negatively correlated` with average temperature, meaning the correlation between wind speed and electricity consumption was mostly likely rooted in temperature differences instead of much to do with wind speed itself.

### Rise in Consumption
Lastly, we investigated the rise in consumption in 2016-2017 by looking at enrollment numbers online. There was about a `5.7%` increase in enrollment (20 people), but that isn't likely to cause such a significant increase in consumption. We haven't yet found anything else that indicates a reason for why this occured, but that is definitely a consideration for future work.

## Which features are the most important?
We were also curious to see which features were most relevant in our target variable, `Total Cons. (kwh)`. So, we merged all the data we had available -- climate and utility -- to create a single dataframe for analysis. 

In [ ]:
from functions_manage_data import merge_all_df

df_all_data = merge_all_df([df_avg_temp, df_avg_wind, df_total_prcp])

Here's what the dataframe looked like after the processing:

In [ ]:
display(HTML(df_all_data.head(10).to_html()))

To analyze which features were the most influential predictors of the total consumption of electricity, we employed the analysis of variance (ANOVA) F-test.

To give a little bit of context, the ANOVA F-test statistic is defined as 
- the ratio of mean square error to regression mean square
    - **mean square error**: the mean of squares of the error between the observed and the predicted value (in our case, generated using linear regression model)
    - **regression mean square**: the mean of squares of difference between the fitted value and the mean of observed values. 

Or alternatively, as
- the ratio of **variation between groups** to **variation within the group**

Another important statistic we obtained was the p-value. The **p-value** is an indicator that if below a certain threshold, ⍺, the result of the statistic is significant. A common value for ⍺ is 0.05. 

A higher F-statistic, combined with a p-value below ⍺, could be interpreted as having greater influence on the target variable.
In our case, the target variable was the total monthly electricity consumption and the features were: 
- monthly total precipitation
- month of year
- time of peak demand
- monthly average wind speed
- monthly average temperature
- total monthly electricity cost

In [ ]:
from functions_f_test import feature_selection

## Separating the features and the target
features = df_all_data.drop(columns=["total_consumption", "year-month"])
target = df_all_data["total_consumption"]

df_results = feature_selection(features, target)

Here are the results:

In [ ]:
display(HTML(df_results.to_html()))

To visualize the results, we made a bar plot that shows the F-Scores of different features. 

In [ ]:
from functions_f_test import plot_f_test

plot_f_test(df_results)

#### Takeaways from the F-Test

First and foremost, it's crucial to take peek at the p-values of the features since it'll inform us about the validity of the F-scores. It's notable that `Monthly Total Precipitation` has a p-value of about 0.8 which indicates that there's a very chance that the observed outcome is a result of chance. But otherwise, all the features have p-value of less than 0.05, so we can use the results of the F-test to gauge the influence of the feature on our target, `Total Monthly Electricity Consumption`. 

Although some might say that including the `Total Monthly Electricity Cost` as a feature is irrelevant for F-test with `Total Monthly Electricity Consumption` as the target, we want to see how the `Monthly Average Temperature` would compare to it as an indicator. As expected, `Total Monthly Electricity Cost` shows the highest F-score, but surprisingly, `Monthly Average Temperature` stands as a strong indicator to electricity consumption even compared to the electricity cost -- its F-score is close to half of that of `Total Monthly Electricity Cost`. `Monthly Average Wind Speed` was the next strongest indicator, which was in line with the prior investigation through the correlation coefficients. `Time of Peak Demand` and `Month of Year` were much weaker indicators in comparison. 


## Summarized Intrepretation

Through our statistical analyses and plots, we found that the weather type correlating most with Olin’s total monthly electricity consumption was Boston's monthly average temperature. These results quantitatively support our initial hypothesis that electricity consumption is most affected by Olin’s heating and cooling system during different seasons of the year. 


Wind speed is the next most correlated dataset; however, as we found in our graph of Average Wind Speed vs Average Temperature, wind speed is highly correlated with temperature, which could explain the correlation with the electricity consumption. Therefore, wind speed may not be directly influential to electricity consumption. 


Precipitation is barely correlated with electricity consumption, maybe even close to random chance within our sample of dataset. 


## Conclusion 

Overall, this project has given us an insight into the impact of our local climate on the energy consumption. It also provides a perspective on the significance of climate change with extremely relevant and localized context. We quantitatively found out that temperature is an outstanding indicator of Olin's electricity consumption. Furthermore, with the knowledge that the cooling towers at Olin operate with electricity, we can infer that one of the biggest contributors to Olin’s electricity consumption is the AC -- in other words, minimizing our use of cooling units will save the most electricity. On a different note, it also suggests that further installation of energy meters on Olin's campus at a building-level, instead of a campus-level meter, would help generate much more data-based insights as such. We could then use the insights to determine biggest factors in energy consumption and point sustainability efforts into projects with greatest capacity for impact. 
